In [1]:
# !conda install --yes tensorflow
# !conda install --yes keras
# !conda install --yes scikit-learn
# !conda install --yes opencv
# !conda install --yes pillow
# !conda install --yes numpy
# !conda install --yes matplotlib
# !conda install --yes theano

In [2]:
# Importing libraries
import tensorflow as tf
# import tensorflow_datasets as tfds
# tf.enable_eager_execution()
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils

from sklearn import metrics
from sklearn.utils import shuffle
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from numpy import *
import os
import cv2
import random
from PIL import Image
import theano

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
# Dataset paths
train_path = "Fast Food Classification V2\Train"
valid_path = "Fast Food Classification V2\Valid"
test_path = "Fast Food Classification V2\Test"

In [4]:
# Important informations
IMG_SIZE = 256
CATEGORIES = sorted(os.listdir(train_path))
CATEGORIES

['Baked Potato',
 'Burger',
 'Crispy Chicken',
 'Donut',
 'Fries',
 'Hot Dog',
 'Pizza',
 'Sandwich',
 'Taco',
 'Taquito']

In [5]:
# Creating X_train and y_train
training = []
for category in CATEGORIES:
    path = os.path.join(train_path, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training.append([new_array, class_num])

random.shuffle(training)

In [ ]:
X_train = [features for features, label in training]
y_train = [label for features, label in training]
# for features, label in training:
#     X_train.append(features)
#     y_train.append(label)
# reshape for the input format expected by keras (256, 256, 3) - 3 channels
X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 3).astype('float32')
# normalizing the pixel values
X_train = X_train / 255.0

In [7]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 10)
print(y_train[100])
print(shape(y_train))

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(15000, 10)


In [8]:
# Creating X_valid and y_valid
validation = []
for category in CATEGORIES:
    path = os.path.join(valid_path, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        validation.append([new_array, class_num])

random.shuffle(validation)

In [ ]:
X_valid = [features for features, label in validation]
y_valid = [label for features, label in validation]
# for features, label in validation:
#     X_valid.append(features)
#     y_valid.append(label)
# reshape for the input format expected by keras (256, 256, 3) - 3 channels
X_valid = np.array(X_valid).reshape(-1, IMG_SIZE, IMG_SIZE, 3).astype('float32')
# normalizing the pixel values
X_valid /= 255

In [10]:
from keras.utils import np_utils
y_valid = np_utils.to_categorical(y_valid, 10)
print(y_valid[100])
print(shape(y_valid))

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
(3500, 10)


In [14]:
# Training model
img_rows, img_columns = 256, 256
img_channels = 3
batch_size = 16     # maybe use 64 or 128 so its faster
nb_epochs = 5       # maybe use 3 so its faster
# nb_classes = 10
# nb_filters = 32
# nb_pool = 2
# nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(img_rows, img_columns, img_channels)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_valid, y_valid))

Epoch 1/5
938/938 [==============================] - ETA: 0s - loss: 2.2262 - accuracy: 0.19 - 1646s 2s/step - loss: 2.2262 - accuracy: 0.1933 - val_loss: 2.0736 - val_accuracy: 0.2780
Epoch 2/5
938/938 [==============================] - 1788s 2s/step - loss: 1.8251 - accuracy: 0.3778 - val_loss: 1.8510 - val_accuracy: 0.3809
Epoch 3/5
938/938 [==============================] - 1812s 2s/step - loss: 1.1492 - accuracy: 0.6199 - val_loss: 2.0258 - val_accuracy: 0.4020
Epoch 4/5
938/938 [==============================] - 1779s 2s/step - loss: 0.5297 - accuracy: 0.8294 - val_loss: 2.5481 - val_accuracy: 0.4003
Epoch 5/5
938/938 [==============================] - 1683s 2s/step - loss: 0.2551 - accuracy: 0.9218 - val_loss: 3.8223 - val_accuracy: 0.3837


The trained model above does not have a good generalization, but it will do for my purpose

In [19]:
# Saving trained model in both the formats SavedModel and HDF5
model.save('saved_model/model1')
model.save('model1_h5.h5')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/model1\assets


In [15]:
# Creating X_test and y_test
testing = []
for category in CATEGORIES:
    path = os.path.join(test_path, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        testing.append([new_array, class_num])

random.shuffle(testing)

In [16]:
X_test = [features for features, label in testing]
y_test = [label for features, label in testing]
# for features, label in testing:
#     X_test.append(features)
#     y_test.append(label)
# reshape for the input format expected by keras (256, 256, 3) - 3 channels
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 3).astype('float32')
# normalizing the pixel values
X_test /= 255

In [17]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test, 10)
print(y_test[100])
print(shape(y_test))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(1500, 10)


In [18]:
# Evaluating model performance on the test set
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  3.58614182472229
Test accuracy:  0.41466665267944336
